In [1]:
import numpy as np
import pymc as pm
import matplotlib.pyplot as plt

In [2]:
def preprocess(data, lower=True, stops=True, emojis=True, 
               newlines=True, single_quotes=True, punctuation=True,
               lemmatize=True, tokenize=True):
    import re
    
    if lower:
        def to_lower_text(text):
            return " ".join([word.lower() for word in text.split()])
        
        data = [to_lower_text(text) for text in data]
    
    if stops:
        import nltk;
        nltk.download('stopwords')
        from nltk.corpus import stopwords
        stop_words = stopwords.words('english')
        
        def remove_stopwords(text):
            return " ".join([word for word in text.split() if word.lower() not in stop_words])
        
        data  = [remove_stopwords(text) for text in data]
        
    if emojis:
        def convert_emojis_and_emoticons_to_word(text):
            for emote in UNICODE_EMO:
                text = text.replace(emote, 
                                    " " + 
                                    "_".join(UNICODE_EMO[emote]
                                    .replace(",","")
                                    .replace(":","").split())
                                    + " ")
            for emote in EMOTICONS:
                text = re.sub(u'('+emote+')', 
                              " " + "_".join(EMOTICONS[emote].replace(",","").split()) + " ", 
                              text)

            return text
        data = [convert_emojis_and_emoticons_to_word(text) for text in data]
        
    if newlines:
        data = [re.sub('\s+', ' ', text) for text in data]
        
    if single_quotes:
        data = [re.sub("\'", "", text) for text in data]
        
    if punctuation:
        data = [re.sub('[^\w\s]','', text) for text in data]
        
    if lemmatize:
        import spacy
        import en_core_web_sm
        nlp = en_core_web_sm.load()
        allowed = ['NOUN', 'ADJ', 'VERB', 'ADV']
        data_lemmatized = []
        for text in data:
            data_lemmatized.append(" ".join([token.lemma_ for token in nlp(text) if token.pos_ in allowed]))
        data = data_lemmatized
        
    if tokenize:
        data = [text.split() for text in data]
        
    return data

## Data
For this task we import Reuters articles from the nltk corpus and sampling 10 random articles which we then preprocess by lowercasing, eliminating anything which is not a word, eliminating stopwords, lemmatizing and converting each document string to a list of word strings.

In [16]:
# import data
import random
import nltk
nltk.download('reuters')
from nltk.corpus import reuters
reuters_training_docs = [reuters.raw(fileid) for fileid in reuters.fileids() if 'training' in fileid]
data = random.sample(reuters_training_docs, 10)
data = preprocess(data, emojis=False)

[nltk_data] Downloading package reuters to
[nltk_data]     C:\Users\eugen\AppData\Roaming\nltk_data...
[nltk_data]   Package reuters is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\eugen\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Sanitization
The next block should be run only if a sanitization check is desired. It will overwrite the data variable so that the algorithm runs on dummy_docs.

In [53]:
dummy_docs =  [["aaa", "bbb", "aaa"],
                ["bbb", "aaa", "bbb"],
                ["aaa", "bbb", "bbb", "aaa"],
                ["uuu", "vvv"],
                ["uuu", "vvv", "vvv"],
                ["uuu", "vvv", "vvv", "uuu"]]
data = dummy_docs

## Data representation
For representing the words we assign an id to each word incrementally and then transform the words in the documents to their corresponding id.

In [5]:
data = [doc for doc in data if doc]

In [54]:
data = dummy_docs

In [55]:
tokens = [word for doc in data for word in doc]

In [56]:
tokens = {word: idx for idx, word in enumerate(set(tokens))}

In [57]:
data_ids = [[0 for j in range(len(data[i]))]
                                    for i in range(len(data))]
for i in range(len(data)):
    for j in range(len(data[i])):
        data_ids[i][j] = tokens[data[i][j]]

## Initialization

In [70]:
# set number of topics
K = 2

In [71]:
V = len(tokens)

In [72]:
M = len(data_ids)

In [73]:
betas = np.ones(V)
alphas = np.ones(K)

## Modelling

$\theta_m$ is the distribution of topics for document m (the probabilities of document m belonging to each topic). - chosen randomly ~ $Dirichlet(\alpha)$

In [74]:
thetas = pm.Container([pm.CompletedDirichlet('theta_%i'%i, pm.Dirichlet('prob_theta_%i'%i, theta=alphas)) for i in range(M)])

$\phi_k$ is the distribution of words in topic k (the probability of each word belonging to a given topic k) chosen randomly ~ $Dirichlet(\beta)$

In [75]:
phis = pm.Container([pm.CompletedDirichlet('phi_%i'%i, pm.Dirichlet('prob_phi_%i'%i, theta=betas)) for i in range(K)])

$z_{m,n}$ is the topic distribution for the word at position n in document m (probabilities of that word belonging to each topic) ~ $Multinomial(\theta_m)$

In [76]:
z = pm.Container(
        [pm.Categorical('z_%i'%m, p=thetas[m], size=len(doc), value=np.random.randint(K, size=len(doc)))
        for m, doc in enumerate(data_ids)])

$w_{m, n}$ draws the actual word from the word distribution ($\phi$) of the topic selected by $z_{m, n}$

In [78]:
w = pm.Container(
        [pm.Categorical('w_%i_%i'%(m,i), p=pm.Lambda('phi_%i_%i'%(m,i), lambda z=z[m][i], phis=phis: phis[z]),
                      value=data_ids[m][i], observed=True)
        for m, doc in enumerate(data_ids) for i in range(len(doc))])

## Sampling

In [83]:
model = pm.Model([thetas, phis, z, w])
mcmc = pm.MCMC(model)
mcmc.sample(400000, 100000, 1)

 [-----------------100%-----------------] 400000 of 400000 complete in 252.8 sec

## Results

In [80]:
## Probabilities of documents belonging to each topic
for m in range(M):
    print(mcmc.trace('theta_%i'%m)[-1])

[[0.58698264 0.41301736]]
[[0.07276805 0.92723195]]
[[0.50812383 0.49187617]]
[[0.55891186 0.44108814]]
[[0.86030168 0.13969832]]
[[0.17709168 0.82290832]]


In [82]:
import numpy as np
## Distribution in each topcic of the 5 most important words
words = list(tokens.keys())
for k in range(K):
    word_distrib_in_topic = mcmc.trace('phi_%i'%k)[-1][0]
    sorted_indexes = np.argsort(word_distrib_in_topic)[::-1]
    sorted_words_distrib = np.sort(word_distrib_in_topic)[::-1]
    most_important_indexes = sorted_indexes[:5]
    most_important_words_distrib = sorted_words_distrib[:5]
    print("Topic " + str(k) + ":")
    for i in range(len(most_important_words_distrib)):
        print(words[most_important_indexes[i]], str(most_important_words_distrib[i]))

Topic 0:
vvv 0.35338176319485654
aaa 0.2721747008617533
bbb 0.1927091449598438
uuu 0.1817343909835463
Topic 1:
vvv 0.7476833240860259
bbb 0.13176858584884615
uuu 0.1103854632837634
aaa 0.010162626781364592


## Topic based similarity between documents
- Essentially, what needs to be done is to find a distance between documents, given their topic/topic distribution
- In the most basic form this could be done by comparing their topics positionally. Having the same topic in the same position in the distribution should make them more similar. This has gaps and wouldn't be well defined, however.
- Upon futher research, we find that there are well defined distance measures for probability distributions. For example the Kullback-Leibler (KL) distance.  
The KL distance measures dissimilarity between to probabilities p and q according to the formula:  
$KL(p, q) = \sum_{i=1}^{T} p_i log \frac{p_i}{q_i}$  
Swapping $p$ and $q$ for $\theta_m$ and $\theta_n$, for 2 documents m and n, we can measure the similarity between 2 documents' topic distributions and so how similar the documents are in terms of their topic.

In [49]:
m = 1
n = 3
theta_m = np.array(mcmc.trace('theta_%i'%m)[-1][0])
theta_n = np.array(mcmc.trace('theta_%i'%n)[-1][0])

KL = np.sum(theta_m * np.log(theta_m/theta_n))

In [50]:
KL

1.7442661912041693

In [51]:
theta_m

array([0.05358214, 0.02297803, 0.11185776, 0.6623711 , 0.14921098])

In [52]:
theta_n

array([0.05767892, 0.02716468, 0.62786727, 0.03111244, 0.25617669])

## New documents
For new documents there are 2 or 3 possible approaches
- First, and probably most accurate, would be adding the new documents to the corpus and running the model again.
- Second, which is more time and resource efficient, would be considering what the model knows as "observed" or "fixed" and only sampling for new documents, so topic assigments for old documents stay the same.  
- However, we have $p_w^i$ the probability of word w being in topic i, and given we have N words in the new document, we can calculate the distribution for each topic, considering words are independent given a topic:  
For each topic i, we compute $\prod_{j=1}^{N} p^i_{w_j}$ OR in log: $\sum_{j=1}^{N} log p^i_{w_j}$  
This would ease computations even further, but there would be problems with out-of-vocabulary words.
